<a align="center">
    <img src="HajimeNoIppo.jpg" width="800">
</a>

<font color='green' size= 6 ><b>Install the following library if not already installed</b></font>

##### Pandas: %pip install pandas

##### Python Dotenv: %pip install python-dotenv

##### Requests: %pip install requests

##### Beautiful soup: %pip install bs4

##### MySQL Connector: %pip install mysql-connector-python

##### Selenium: %pip install selenium

<font color='green' size= 6 ><b>Import Library necessary for the Project and define the variables</b></font>

In [1]:
# importing os ans sys modules
import os
import sys

# Navigate to project directory to import modules

sys.path.insert(0, os.path.dirname(os.getcwd()))

# Importing function create from Manga_function.py
from Functions.Manga_function import write_to_file, create_soup, extract_date, extract_episod, extract_link, extract_title 

# MySQL connector
import mysql.connector

# Pandas
import pandas as pd

# importing os module for environment variables
import os

# importing necessary functions from dotenv library
from dotenv import load_dotenv

# loading variables from .env file
load_dotenv() 

True

In [2]:
# Define the variables

host = os.getenv("MYSQL_HOSTNAME")
user = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
database = "manga"
table_name = "HajimeNoIppo"
manga_name = "Hajime No Ippo"
csv_file = "C:/Users/yacbo/Documents/Data_Engineer/Training/Manga_Project/HajimeNoippo/HajimeNoippo.csv"
log_file = "C:/Users/yacbo/Documents/Data_Engineer/Training/Manga_Project/HajimeNoippo/HajimeNoippo.txt"
url = os.getenv("URL")
number_tot_ep = 76

<font color='green' size= 6 ><b>Run "Manga_init.ipynb"</b></font>

In [3]:
%run C:\Users\yacbo\Documents\Data_Engineer\Training\Manga_Project\Manga_init.ipynb

<font color='green' size= 6 ><b>Extract and Transform the data</b></font>

In [4]:
if number_ep_to_extract == 0 :
    write_to_file(log_file, f"Program stopped because there is no data to extract")
    raise SystemExit("System stop because there is no episod to extract")

In [6]:
# find the first link to extract
if number_ep_database == 0: # if no episod in database take the episod 1 link
    link  = url
else: # if there is some episod in database extract the link from the last episod extracted
    # Extract link of the last episod in database
    try:
            # Execute a query
        cursor.execute(f"SELECT link from {table_name} order by Episod desc limit 1")
        link_database = cursor.fetchall()
        link_database = str(link_database[0][0])
    except mysql.connector.Error as err:
        write_to_file(log_file, f"There is an error : {err}")
        # Close the database cursor and connection
        if cursor:
            cursor.close()
        if cnx:
            cnx.close()

    # Create a beaufiful soup from this link and extract the link of the following episod       
    soup = create_soup(link_database)
    link = soup.find_all('a', class_='ipc-icon-button sc-3f4e3993-3 iasCTO ipc-icon-button--baseAlt ipc-icon-button--onBase')[1].attrs['href']
    link = extract_link(link)

In [7]:
# create lists
season_list = []
episod_season_list = []
episod_list= []
title_list = []
rating_list = []
day_list= []
month_list= []
year_list = []
link_list = []

link_list.append(link)

# Extract data for each episod
for i in range(number_ep_to_extract):

    try:

        write_to_file(log_file, f"Extracting episod {number_ep_database+1+i} from {link}")
        soup = create_soup(link)
        
        # Season and Episod
        ep = soup.find('div', class_="sc-3f4e3993-0 fYpskP").text
        season,episod_season = extract_episod(ep)
        season_list.append(season)
        episod_season_list.append(episod_season)
        
        # Title
        title = extract_title(soup.find('span', class_="hero__primary-text").text)
        title_list.append(title)

        #Rating
        rate = soup.find('span', class_="sc-c4ffe080-1 iQZtLP").text.replace(",",".")
        rating_list.append(float(rate))

        # Date
        date = soup.find_all('li', class_="ipc-inline-list__item")
        day,month,year = extract_date(date)
        day_list.append(day)
        month_list.append(month)
        year_list.append(year)

        #Link 
        if i == 0 and number_ep_database == 0: 
            link = soup.find('a', class_='ipc-icon-button sc-3f4e3993-3 iasCTO ipc-icon-button--baseAlt ipc-icon-button--onBase').attrs['href']
        else:
            link = soup.find_all('a', class_='ipc-icon-button sc-3f4e3993-3 iasCTO ipc-icon-button--baseAlt ipc-icon-button--onBase')[1].attrs['href']

        if i < number_ep_to_extract - 1:        
            link = extract_link(link)
            link_list.append(link)

        i = i+1

    except IndexError:
        pass         

    except WebDriverException as E: 
        write_to_file(log_file, f"Extracting episod {number_ep_database+1+i} from {link} failed")
      
episod_list = episod_season_list # As there is only one season

<font color='green' size= 6 ><b>Load the data</b></font>

In [8]:
# Create DataFrame of the data
data = {"Episod":episod_list,"Season":season_list,"Episod_season":episod_season_list, "Title":title_list, "Day":day_list, "Month":month_list, "Year":year_list, "Rate":rating_list, "link":link_list}
df = pd.DataFrame(data)

# Check if all the data have been successfully extracted
if df.shape[0] == number_ep_to_extract:
    write_to_file(log_file,f"Extracting {number_ep_to_extract} episod successfuly")
else:
    write_to_file(log_file,f"Extraction failed")

In [9]:
# Load the data into a csv file
df.to_csv(csv_file,index=False,mode= "a")

In [10]:
# Load the data into MySQL database

write_to_file(log_file,f"Sending {len(df)} rows to MySQL table '{table_name}'.")
try:
    # Create a list of column names from the DataFrame
    columns = ', '.join(df.columns)

    # Create a placeholders string for the SQL query
    placeholders = ', '.join(['%s'] * len(df.columns))

    # Create the SQL query
    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

    # Execute the SQL query with the DataFrame values
    cursor.executemany(sql, df.values.tolist())

    # Commit the changes to the database
    cnx.commit()

    write_to_file(log_file,f"Successfully sent {len(df)} rows to MySQL table '{table_name}'.")
except mysql.connector.Error as e:
    print(f"Error sending data to MySQL: {e}")
finally:
    # Close the database cursor and connection
    if cursor:
        cursor.close()
    if cnx:
        cnx.close()